Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 08

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, June 12, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Assignment 1: Multilayer Perceptron (MLP) [10 Points]

Last week you implemented a simple perceptron. This week we already provide some basic perceptron which you will adjust to build network from it.

In [ ]:
import numpy as np
class SimplePerceptron:
    """
    A simple perceptron implementation.
    """

    def __init__(self, dimensions=100, epsilon=0.03):
        """
        Initializes the perceptron. Creates dimensions + 1
        random weights (the additional weight is the bias.)

        Args:
            dimensions  the data dimensionality N
            epsilon     the learning rate
        """
        self.w = np.random.rand(dimensions + 1)
        self.epsilon = epsilon
        
    def activation(self, X):
        """
        The activation function. Prepends a 1 to X for the
        bias and calculates the activation function of the 
        perceptron.

        Args:
            X           the data point, should be a numpy
                        arary or a 1xN numpy matrix

        Returns:
            True  if the activation of X is bigger than 0
            False elseal
        """
        return np.append(1, X) @ self.w > 0

    def train(self, X, t):
        """
        Trains the perceptron. Adjusts the weights according to 
        the learning rate and the error between the activation and t.

        Args:
            X           the data point, should be a numpy
                        arary or a 1xN numpy matrix
            t           the label for this data point should be
                        True or False
        """
        self.w += self.epsilon * (t - self.activation(X)) * np.append(1, X)

# Generate some data.
N = 1000
dim = 3
D = np.random.rand(1000, dim)
# Label data: sum should be > 0.8 * dim
D = np.hstack((D, np.matrix(np.sum(D, 1) > 0.8 * dim).T))

# Instantiate a Perceptron.
perceptron = SimplePerceptron(D.shape[1] - 1)

# Train the perceptron for several epochs.
epochs = 20
sample_size = 100
for epoch in range(epochs):
    for sample in range(sample_size):
        sample_data = D[np.random.choice(range(N), replace=False),:]
        for data in sample_data:
            x = data[0,0:-1]
            t = data[0,-1]
            perceptron.train(x, t)

# Test the perceptron on all data.
error = 0
for data in D:
    error += np.abs(data[0,-1] - perceptron.activation(data[0,0:-1])) / N
print("The perceptron classifies {:.2%} of the data correctly.".format(1 - error))

In [ ]:
class Perceptron:

    def __init__(self, dimensions=100, epsilon=0.03):
        self.w = np.random.rand(dimensions + 1)
        self.epsilon = epsilon
        self.y = None

    def activation(self, X):
        self.y = ...
        return self.y

    def train(self, X, t, delta):
        pass

In [ ]:
class MultiLayerPerceptron:
    def __init__(self, neurons_per_layer):
        

## Assignment 2: MLP and RBFN [10 Points]

This exercise is aimed at deepening the understanding of Radial Basis Function Networks and how they relate to Multilayer Perceptrons. Not all of the answers can be found directly in the slides - so when answering the (more algorithmic) questions, first take a minute and think about how you would go about solving them and if nothing comes to mind search the internet for a little bit. If you are interested in a real life application of both algorithms and how they compare take a look at this paper: [Comparison between Multi-Layer Perceptron and Radial Basis Function Networks for Sediment Load Estimation in a Tropical Watershed](http://file.scirp.org/pdf/JWARP20121000014_80441700.pdf)

### Radial Basis Function Networks

![Schematic of a RBFN](RBFN.png)

What are radial basis functions?

What is the structure of a RBFN? You may also use the notion from the above included picture.

How would one initialize such a network? (Hint: make use of other algorithms already covered in the lecture)

How is a RBFN trained?

### Comparison to the Multilayer Perceptron

How can classification in both networks be visualized?

When would you use a RBFN instead of a Multilayer Perceptron?